# Trading EDA

To install conda and binance packages to this notebook uncomment the code below

In [ ]:
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
%pip install binance-connector==1.13.0
%pip install pandas
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Init binance client

In [ ]:
from binance.spot import Spot as Client
import os
import sys
import yaml

# Load 
def load_config():
    """
    Load config from cfg folder respecting the order: defaults, app.yaml, environment vars
    """
    # Defaults
    cfg_dir="./../biml/cfg"
    default_cfg_path = f"{cfg_dir}/app-defaults.yaml"
    with open(default_cfg_path, "r") as appdefaults:
        config = yaml.safe_load(appdefaults)

    # Custom config, should contain custom information,
    cfg_path = f"{cfg_dir}/app.yaml"
    if os.path.exists(cfg_path):
        with open(cfg_path) as app:
            config.update(yaml.safe_load(app))
    else:
        sys.exit(
            f"Config {cfg_path} not found. Please copy cfg/app-defaults.yaml to {cfg_path} "
            f"and update connection info there.")

    # Enviroment variabless
    config.update(os.environ)
    return config


# Create binance client
config=load_config()
key,secret,url=config["biml.connector.key"],config["biml.connector.secret"],config["biml.connector.url"]
client = Client(key=key, secret=secret, base_url=url, timeout=10)


Get account info

In [ ]:
account=client.account()
print(account)


Get trade bot orders from sqlite db

In [ ]:
def get_trades():
    import sqlite3
    db_path="../data/biml.db"

    with sqlite3.connect(db_path) as conn:
        trades=pd.read_sql_query("SELECT * FROM trade ORDER BY open_time",conn,
                                 parse_dates=["open_time", "close_time"])#.set_index("open_time", drop=False)
    trades["profit"] = trades["close_price"]-trades["open_price"]
    return trades

trades=get_trades()
trades.tail()

Get opened trades

In [ ]:
trades[trades.close_time.isna()].tail()

In [ ]:
# # Get orders
# import pandas as pd
# import numpy as np
# symbol="BTCUSDT"

# # Get orders, but market orders have price=0
# orders=pd.DataFrame(client.get_orders(symbol))
# if not orders.empty: orders=orders.astype({'time': 'datetime64[ms]','updateTime':'datetime64[ms]','price':'float64','origQty':'float64'}).set_index('time',drop=False)

# # Get trades and pull orders' prices from there
# trades=pd.DataFrame( client.my_trades(symbol))
# if not trades.empty: trades=trades.astype({'time': 'datetime64[ms]', 'price':'float64'}).set_index('time',drop=False)    
# orders=orders.drop("price",axis=1).merge(trades[["orderId","price"]],on="orderId")

# # Calculate profit of closing orders
# orders["delta_price"]=orders["price"].diff()
# orders["profit"] = orders["delta_price"]*orders["origQty"]
# orders.loc[~orders["type"].isin(["LIMIT_MAKER", "STOP_LOSS_LIMIT"]),["profit"]]=np.nan

# orders["time"] = orders["updateTime"]
# orders.set_index("time", drop=False, inplace=True)
# orders.tail()


Look at opened stop loss and take profit orders

In [ ]:
symbol="BTCUSDT"
orders_opened=pd.DataFrame(client.get_open_orders(symbol))
orders_opened.tail()

Get candles from binance

In [ ]:
def get_last_candles():
    candle_columns = ["open_time", "open", "high", "low", "close", "vol", "close_time", "quote_asset_volume",
                      "number_of_trades", " taker_buy_base_asset_volume", "taker_buy_quote_asset_volume",
                      "ignore"]    
    end_date=trades.open_time.max() if not trades.empty else datetime.now()
    end_time_millis=end_date.value//1000000
    raw_candles = client.klines(symbol=symbol,
                                    interval="1m",
                                    #limit=10,
                                    limit=24*60*60,
                                    endTime=end_time_millis
                               )
    df = pd.DataFrame(raw_candles, columns=candle_columns)
    df["open_time"] = pd.to_datetime(df["open_time"], unit='ms')
    df["close_time"] = pd.to_datetime(df["close_time"], unit='ms')
    df.set_index("open_time", drop=False, inplace=True)
        # Convert strings to float prices
    df[["open", "high", "low", "close"]] = df[["open", "high", "low", "close"]].astype(float)
    return df

def clean_candles(df):
    """ Remove outliers """
    q_low = df["low"].quantile(0.01)
    q_high  = df["high"].quantile(0.99)
    df=df[df["high"]< q_high]
    df=df[df["low"] > q_low ]
    return df

candles = get_last_candles()
candles = clean_candles(candles)
candles.tail()

In [ ]:
candles.tail()

## EDA

### Look at trades quantity cumulative sum

## Look at trades on candlestick chart

In [ ]:
# Install a conda package in the current Jupyter kernel
#%conda install -c plotly plotly=5.9.0
# Draw candlechart
import plotly as py
from plotly import graph_objects as go
import pandas as pd
import numpy as np
import datetime

df=candles.copy()
#df=clean_candles(df)
fig = go.Figure(data=[ \
                    go.Candlestick(\
                        x=df.index,\
                        open=df['open'],\
                        high=df['high'],\
                        low=df['low'],\
                        close=df['close'])
                     ])

# Customize ay to distribute captions by y so they do not overlap by x
ay,aymax,aystep=0,50,10
#start_time=df.open_time.min()
# Annotate buy, sell on the chart
for i,trade in trades[(~trades.open_time.isna()) & (trades.open_time > df.open_time.min())].iterrows():
    ay=(ay+aystep)%aymax
    color='green' if trade['side']=="BUY" else 'red'

    # Draw open order
    open_text='buy' if trade['side']=="BUY"  else 'sell'
    open_hover=f"{trade['open_time']} {open_text} at {str(trade['open_price'])}"
    fig.add_annotation(x=trade['open_time'], \
                       y=trade['open_price'],\
                       text=open_text,\
                       hovertext=open_hover,\
                       showarrow=True,arrowhead=5,arrowcolor=color,ay=-ay,arrowwidth=2,
                       font={'color':color}
                      )
for i,trade in trades[(~trades.close_time.isna()) & (trades.close_time > df.close_time.min())].iterrows():

    print("close_time:"+str(trade.close_time.value))
    # Draw close order
    #profit = trade["close_price"]-trade["open_price"] if trade["close_price"] else None
    status= f", profit: {round(trade.profit, 10)}" if trade.profit else ""
    close_text='close buy' if trade['side']=="BUY"  else 'close sell'
    close_hover=f"{trade['close_time']} {close_text} at {str(trade['close_price'])}{status}"
    fig.add_annotation(x=trade['close_time'], \
                       y=trade['close_price'],\
                       text=close_text,\
                       hovertext=close_hover,\
                       showarrow=True,arrowhead=5,arrowcolor=color,ay=-ay,arrowwidth=2,
                       font={'color':color}
                      )

fig.show()


## Profit

In [ ]:
#import matplotlib.pyplot as plt
import plotly.express as px
# Calc profit, considering only closing trades
profit=trades.copy()
profit["cum_profit"] = profit["profit"].cumsum()
profit["color"]=np.where(profit.profit >=0, "green","red")
profit.tail()

#Plot
fig = px.line(profit, x="open_time", y="profit")
fig.show()

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
# Calc profit, considering only closing trades
profit=trades[~trades["profit"].isna()].copy()
#profit["profit"]=profit["price"].diff()*(profit["qty"]/profit["qty"])
profit["cum_profit"] = profit["profit"].cumsum()

#Plot
fig = px.line(profit, y="cum_profit")
fig.show()

## Predicted and actual low/high

In [ ]:
import os
import glob
import pandas as pd
from datetime import timedelta,date
import matplotlib.pyplot as plt
import plotly.express as px

def read_predictions(start_date_inc, end_date_inc):
    """ Read days from start to end from local predictions history"""
    symbol="BTCUSDT"    
    data_dir=f"./../model-dev/PredictLowHighCandlesStrategy/Xy"
    print(f"start date:{start_date_inc}, end date:{end_date_inc}, all inclusive")
    df = pd.DataFrame()
    candles_list=[]
    for n in range(int ((end_date_inc - start_date_inc).days)+1):
        cur_date = start_date_inc + timedelta(n)    
        candles_path=f"{data_dir}/{cur_date}_{symbol}_candles.csv"     
        if not os.path.exists(candles_path): continue
        #print(f"Read {cur_date} candles from {csv_path}")
        cur_candles=pd.read_csv(candles_path,index_col="close_time")
        candles_list.append(cur_candles)
    df = pd.concat(candles_list)
    return df


# Read candles for last 3 days
last_date=trades.open_time.max().date()
first_date=last_date - timedelta(days=0)

df = read_predictions(first_date, last_date)
px.line(df[["low","high","fut_low","fut_high"]], markers=True)


## Look at the account

In [ ]:
print(account)